In [1]:
# Queries: https://peakd.com/hive-191635/@enrique89/technical-report-analysis-of-the-global-presence-of-hive

In [1]:
cfile='usa.txt'
days=36

In [17]:
# Read users
u = []
with open(cfile,'r') as f:
    for l in f:
        if not 'R,' in l:
            if '@' in l:
                u += l.strip().replace('(', '').replace(')', '').replace('@', '').split()
            elif '(' in l:
                u.append(l.split()[0])
            else:
                u.append(l.strip())
                
hacked = []
#with open('steemengineteam_group.txt','r') as f:
#    for l in f:
#        hacked.append(l.strip())
        
ignore = []
#with open('ignore.txt','r') as f:
#    for l in f:
#        ignore.append(l.strip().split()[0])


In [3]:
import pyodbc
import pandas as pd
import datetime

In [12]:
with open('pw.txt') as f:
    for l in f:
        uname, pw = l.split()

cnxn_str = ("Driver={ODBC Driver 18 for SQL Server};"
            "Server=vip.hivesql.io;"
            "Database=DBHive;"
            f"UID={uname};"
            f"PWD={pw};"
            "Encrypt=no")
cnxn = pyodbc.connect(cnxn_str)

In [15]:
if cfile == 'usa.txt':
    searches=["JSON_VALUE(posting_json_metadata,'$.profile.location') like '%usa%'",
             "JSON_VALUE(posting_json_metadata,'$.profile.location') like '%united states%'"]

In [18]:
fromdate = (datetime.date.today()-datetime.timedelta(days=days)).strftime('%Y-%m-%d')
#print(f'From {fromdate}')
fnd=[]
for s in searches:
    r = pd.read_sql(f"""select a.name, a.post_count, JSON_VALUE(posting_json_metadata,'$.profile.location') as loc,
                case when a.json_metadata like '%location%' then JSON_VALUE(json_metadata,'$.profile.location')
                else 'none' end as jm , a.last_post, a.last_vote_time, a.recovery_account
                from Accounts a
                where a.posting_json_metadata like '%"location":%' and ({s})
                    and a.last_post >= '{fromdate}'""", cnxn)
    for i, l in r.iterrows(): 
        n= l['name']
        if n not in u and n not in ignore and n not in fnd:
            print(f"Post: {str(l['last_post'])[:10]} Vote: {str(l['last_vote_time'])[:10]} {l['name']} ({l['loc']}) Rec: {l['recovery_account']}")
            #print(f"{l['name']} ({l['loc']})")
            fnd.append(n)

Post: 2025-11-14 Vote: 1970-01-01 fotolibrepro (usa) Rec: appreciator
Post: 2025-12-06 Vote: 2025-12-06 susanli3769 (likerid:susanli3769) Rec: steem
Post: 2025-12-06 Vote: 2025-12-06 jang (Lithuania/Norway/USA  ) Rec: steem
Post: 2025-11-12 Vote: 2021-12-07 bchainguru (USA) Rec: quello
Post: 2025-11-10 Vote: 1970-01-01 charismaticrebel (USA) Rec: ecency
Post: 2025-12-06 Vote: 2025-12-06 dewabrata (Nusantara) Rec: steemmonsters
Post: 2025-11-24 Vote: 2025-11-24 sioyen (Nusantara) Rec: leo.voter
Post: 2025-12-01 Vote: 2025-03-17 mariesn (USA) Rec: ecency
Post: 2025-12-05 Vote: 2025-12-05 ecwwebsite (United States) Rec: ocd
Post: 2025-12-05 Vote: 2024-08-24 arnewswire (United States) Rec: appreciator
Post: 2025-12-01 Vote: 2025-04-14 slackerman (United States) Rec: oracle-d
Post: 2025-12-06 Vote: 1970-01-01 smailproseo (United States) Rec: appreciator
Post: 2025-11-27 Vote: 2025-11-27 sylregne (United States) Rec: ecency
Post: 2025-11-01 Vote: 2025-11-07 threedotz (United States) Rec: ece

In [11]:
from beem import Hive
from beem.account import Account
from beem.amount import Amount
from beem.blockchain import Blockchain
from beem.comment import Comment
from beem.instance import set_shared_blockchain_instance

tracktags = set(['teamuk','england','scotland','wales','unitedkingdom','greatbritain','uk'])
hive = Hive(node=['https://anyx.io'])#https://hived.emre.sh'])#https://api.deathwing.me', 'https://api.hive.blog'])
set_shared_blockchain_instance(hive)
chain = Blockchain()
current_num = chain.get_current_block_num()
for post in chain.stream(opNames="comment", threading=True, thread_num=5, start=current_num-30000, stop=current_num):
    c = Comment(post)
    if c.is_main_post() and len(post['tags']) and tracktags.intersection(post['tags']):
        author = post['author']
        if author not in u and author not in ignore:
            print(author,post['title'],post['tags'])

Error: Gateway Time-out
Retry RPC Call on node: https://anyx.io (1/5) 

Retrying in 0 seconds

Error: Gateway Time-out
Retry RPC Call on node: https://anyx.io (2/5) 

Retrying in 2 seconds

Error: Gateway Time-out
Retry RPC Call on node: https://anyx.io (3/5) 

Retrying in 3 seconds

Error: Gateway Time-out
Retry RPC Call on node: https://anyx.io (4/5) 

Retrying in 5 seconds

Error: Gateway Time-out
Retry RPC Call on node: https://anyx.io (5/5) 

Retrying in 6 seconds



ValueError: Could not receive dynamic_global_properties!

In [ ]:
'steemingmark' in u

In [9]:
from beem import Hive
from beem.account import Account
from beem.amount import Amount
from beem.blockchain import Blockchain
from beem.comment import Comment
import beem.witness
from beem.instance import set_shared_blockchain_instance

hive = Hive(node=['https://anyx.io'])
print(f'Minnow {hive.vests_to_hp(1E6):.3f}')
print(f'Dolphin {hive.vests_to_hp(1E7):.3f}')
print(f'Orca {hive.vests_to_hp(1E8):.3f}')
print(f'Whale {hive.vests_to_hp(1E9):.3f}')


Minnow 602.818
Dolphin 6028.182
Orca 60281.825
Whale 602818.248


In [ ]:
w=[]
for i in u:
    if len(beem.witness.GetWitnesses([i])):
        w.append(f'@{i}')
        print(i)
print(w)

In [ ]:
l='elmerlin (London @liketu)'
unames = []
if '@' in l:
    unames.append(l.split()[0])
    l = l[l.find('@'):]
    unames += l.strip().replace('(', '').replace(')', '').replace('@', '').split()
elif '(' in l:
    unames.append(l.split()[0])
else:
    unames.append(l.strip())
print(unames)
